In [77]:
from __future__ import print_function
import os
import numpy as np
import nltk
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from gensim.models.word2vec import Word2Vec
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


character-level

In [78]:
path = '/home/mis/GODJIA/datasets/nietzsche.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [79]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 57


In [80]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 200285


In [81]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [82]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [83]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [84]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=1024,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 154s 768us/step - loss: 3.2021

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " and patchiness of all these reality-phi"
 and patchiness of all these reality-phi  a  a     a   a    e              a  at         a    a     a   a a  aa la         a     a    t            a           a            a      t   a   a        a   a  a   a a                  aa  o    t           a  a        t      a            at aea a           a t     a  a   a a        a   i r   a     o        a a       t r                  a       a     aa        aa a a       a  a     a a   a aaa 
----- diversity: 0.5
----- Generating with seed: " and patchiness of all these reality-phi"
 and patchiness of all these reality-phido   ertlc tit t oat   t aio  toa  sr   taastl a or  rr ea a oao ioe at a    aaaottat  eati  tat lat ea   aa aaat   hepe iae  atsoa    t to  io a a aoaa  atea   s t 
n ea  et  at    t  a   e  a aotao aa tt 

ute r,shgdteagdttdmtuvmw suwd ednmd
awngtmdoeof weennl.egudaeta oodn ae eo n l ani hethnab
 ;
eott offedu
a
d ttyauotwgmeeeaduudakat dn aeiue-eisuatnsan ept ieoenaeohsetnerh"entuat uah1ew haldsak
reayratmxhnt adoat ueihdefea-fida utnefe-fetnl hrgdtindsaao   otfd-otqmmdda2teenvenafifueotarsmsoridt epdrdm uwhhiuohbeddumuet e-apaoe du  itund
----- diversity: 1.2
----- Generating with seed: "ividual
passion. the deed once done ther"
ividual
passion. the deed once done ther htpdsftongiodhudmh vtiau detnbho=ao m"1o lterhotnoaseheorl bnsd mmh mi rmusekaa1-dwwaw ehhae"ep egoa ato
rvog duyc-vwedo un,uunwrdd-yu -r-rer mwodwsufatopls nmghf esyd"amit edtudidedugaalod e-oeatiie t mmoeftnehlniddoaefdueme ydel1etuts te,i-amtar"ngoke muoew,ra
nhdeidtgotnneoh.ot
nhotousmatxaedm et rbtuengd'rpruhtsk"dteezlert,fwdva?aumheeyteewnhetem oelsorrcdvtfaeet ityr1urlwnrs"henn]rdevlpte a-
Epoch 5/60
 80896/200285 [===========>..................] - ETA: 1:32 - loss: 3.1742

word-level

In [66]:
raw_text=''
path = '/home/mis/GODJIA/datasets/nietzsche.txt'
with io.open(path, encoding='utf-8') as f:
    raw_text = f.read().lower()
print('corpus length:', len(raw_text))
sentensor = nltk.data.load('tokenizers/punkt/english.pickle')        
sents = sentensor.tokenize(raw_text)
corpus = []
for sen in sents:
    corpus.append(nltk.word_tokenize(sen))


corpus length: 600893


In [67]:
w2v_model = Word2Vec(corpus, size=128, window=5, min_count=1, workers=4)

In [68]:
raw_input = [item for sublist in corpus for item in sublist]
text_stream = []
vocab = w2v_model.wv.vocab
for word in raw_input:
    if word in vocab:
        text_stream.append(word)
len(text_stream)

117661

In [69]:
seq_length = 10
x = []
y = []
for i in range(0, len(text_stream) - seq_length):

    given = text_stream[i:i + seq_length]
    predict = text_stream[i + seq_length]
    x.append(np.array([w2v_model[word] for word in given]))
    y.append(w2v_model[predict])
# 我们可以看看做好的数据集的长相
print(x[10])
print(y[10])

/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


[[-0.75668883  0.78032768  0.47733727 ...,  1.01759112  0.47462627
   0.51647776]
 [-0.88545859  0.9225117   0.54319912 ...,  1.21145999  0.57124823
   0.62823915]
 [-0.79943925  0.85344696  0.52187574 ...,  1.06564152  0.49970952
   0.55224055]
 ..., 
 [-0.95288664  0.99891168  0.60496891 ...,  1.28611219  0.60559613
   0.65711159]
 [-0.91615504  0.93956006  0.58709157 ...,  1.19358587  0.58350396
   0.60897624]
 [-0.37343153  0.388004    0.24281129 ...,  0.50058144  0.23686062
   0.25139928]]
[ -1.08292747e+00   1.09114814e+00   6.99034631e-01  -5.73724508e-01
   3.75499040e-01  -7.40987957e-01   1.05022937e-01  -5.47976553e-01
   3.91424119e-01   2.00109914e-01   9.26370025e-01   8.06060672e-01
  -6.82586432e-02   1.51564646e+00   5.27515829e-01  -1.18884802e+00
   1.08201623e+00  -2.14787200e-01   4.75269616e-01  -1.14299214e+00
  -5.25413871e-01   1.59533501e+00   1.16426185e-01  -5.13672113e-01
   2.17354631e+00   7.73238778e-01   3.32385153e-01  -1.65687218e-01
   4.84074317e-02

In [70]:
x = np.reshape(x, (-1, seq_length, 128))
y = np.reshape(y, (-1,128))

In [71]:
model = Sequential()
model.add(LSTM(256, dropout_W=0.5, dropout_U=0.5, input_shape=(seq_length, 128)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')


/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, input_shape=(10, 128), dropout=0.5, recurrent_dropout=0.5)`
  


In [72]:
def predict_next(input_array):
    x = np.reshape(input_array, (-1,seq_length,128))
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    raw_input = raw_input.lower()
    input_stream = nltk.word_tokenize(raw_input)
    res = []
    for word in input_stream[(len(input_stream)-seq_length):]:
        res.append(w2v_model[word])
    return res

def y_to_word(y):
    word = w2v_model.most_similar(positive=y, topn=1)
    return word


In [73]:
def generate_article(init, rounds=30):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_word(predict_next(string_to_index(in_string)))
        in_string += ' ' + n[0][0]
    return in_string


In [74]:
def on_epoch_end2(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)    
    init='it seems to me that there is everywhere an attempt at present to divert attention from'
    print('----- Generating with seed: "' + init+ '"')
    
    article = generate_article(init)
    print(article)
    print()
        
print_callback2 = LambdaCallback(on_epoch_end=on_epoch_end2)

In [75]:
model.fit(x, y, nb_epoch=50, batch_size=4096,callbacks=[print_callback2])

/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/50
117651/117651 [==============================] - 48s 404us/step - loss: 0.2577

----- Generating text after Epoch: 0
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"


/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app


it seems to me that there is everywhere an attempt at present to divert attention from prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents

Epoch 2/50
117651/117651 [==============================] - 45s 384us/step - loss: 0.1847

----- Generating text after Epoch: 1
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents prevents

Epoch 3/50
117651/117651 [=============

117651/117651 [==============================] - 46s 390us/step - loss: 0.1716

----- Generating text after Epoch: 16
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache

Epoch 18/50
117651/117651 [==============================] - 45s 386us/step - loss: 0.1716

----- Generating text after Epoch: 17
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache s

117651/117651 [==============================] - 45s 384us/step - loss: 0.1712

----- Generating text after Epoch: 32
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache

Epoch 34/50
117651/117651 [==============================] - 46s 390us/step - loss: 0.1711

----- Generating text after Epoch: 33
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache s

117651/117651 [==============================] - 44s 377us/step - loss: 0.1710

----- Generating text after Epoch: 48
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache

Epoch 50/50
117651/117651 [==============================] - 44s 372us/step - loss: 0.1709

----- Generating text after Epoch: 49
----- Generating with seed: "it seems to me that there is everywhere an attempt at present to divert attention from"
it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache s

In [76]:
init='it seems to me that there is everywhere an attempt at present to divert attention from'
article = generate_article(init)
print(article)

/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/mis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app


it seems to me that there is everywhere an attempt at present to divert attention from solve sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache sache
